In [29]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras.models import load_model

import pickle

In [30]:
model = load_model('ann_model.h5')

In [31]:
with open('labelEncoder_gender.pkl', 'rb') as file:
    labelEncoder_gender = pickle.load(file)

with open('oneHotEncoder_geo.pkl', 'rb') as file:
    oneHotEncoder_geo = pickle.load(file)

with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

In [32]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [33]:
## One-hot encode the Geography
geo_encoded = oneHotEncoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=oneHotEncoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/denesh/Desktop/Learning/Generative-AI-With-Langchain-and-Huggingface/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [34]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [35]:
input_data_df.drop(['Geography'], axis =1, inplace=True)

input_data_df = pd.concat([input_data_df, geo_encoded_df], axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
input_data_df['Gender'] = labelEncoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
input_data_scaled = scalar.transform(input_data_df)
input_data_scaled

/Users/denesh/Desktop/Learning/Generative-AI-With-Langchain-and-Huggingface/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-6.61982076,  0.72650913, -3.75643711, -1.95944749, -1.25600183,
        -1.22232579, -0.17655887,  0.8563161 , -1.71828828,  0.94058025,
        -1.90736859, -1.8810149 ]])

In [40]:
pred = model.predict(input_data_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.00118521]], dtype=float32)

In [43]:
pred_prob = pred[0][0]

if pred_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [ ]:
oneHotEncoder_geo.categories_[0]

'France'

'Female'